In [1]:
import sys 
sys.path.append("..") 
from pubmedkit.load_pubmed_file import load_pubmed_file
import pandas as pd

In [2]:
records = load_pubmed_file("../data/pubmed_Arabidopsis_ChIP.txt",output_type='dict')

In [3]:
# records

In [4]:
from langchain_community.llms.ollama import Ollama
from langchain_core.prompts import PromptTemplate
from tqdm import tqdm

In [5]:
planttemplate = """ 

Transcription factor (TF)  is a protein that controls the rate of transcription of genetic information from DNA to messenger RNA, by binding to a specific DNA sequence.   Chromatin immunoprecipitation (ChIP) is a type of immunoprecipitation experimental technique used to investigate the interaction between proteins and DNA in the cell. It aims to determine whether specific proteins are associated with specific genomic regions, such as transcription factors on promoters or other DNA binding sites, and possibly define cistromes. Arabidopsis or Arabidopsis thaliana is model organism for studying the function of transcription factors. Arabidopsis thaliana is not name of the TF.

Context:{context}.

Question: If the above content related to research in transcription factor of Arabidopsis, please give name of Transcription factor. If don't find name of TF, just say that you don't know or don't find TF's name, don't try to make up an answer.

"""


In [6]:
llm = Ollama(
    model= 'qwen2.5:14b',
    base_url='http://localhost:11434',  # 可以填 192.168.127.87
    temperature=0
)

In [7]:

for pmid in records:
    abstract = records[pmid]['abstract']
    prompt = PromptTemplate(input_variables=['text'], template=planttemplate)
    chain = prompt | llm
    response = chain.invoke(abstract).rstrip('\n')
    print(records[pmid]['pmid'],response)
    records[pmid]['plant_response'] = response

25215497 I don't know. The text mentions "Arabidopsis" as a model organism for studying transcription factors, but it does not mention the specific name of a transcription factor.
30909785 I found several names of transcription factors mentioned in the content:

1. TGA9 (TGACG motif-binding protein 9)/AT1G08320
2. AP2-EREBP
3. ARF
4. ATF4
5. FOXO
6. TFEB
36403487 The name of the Transcription Factor is WRKY29.
32445409 The name of the Transcription Factor is ABI3 (ABSCISIC ACID INSENSITIVE3).
37382898 The name of the Transcription Factor mentioned is BBX14.
26817596 I don't know. The text does not mention the name of a specific transcription factor. It only mentions Arabidopsis thaliana as a model organism for studying transcription factors in general.
35806432 I found it!

The name of the Transcription Factor is: AtOFP1 (Arabidopsis thaliana Ovate family protein 1)
23340744 The name of the Transcription Factor is ORE1 (ANAC092).
35968123 I found it!

The names of the transcription fac

In [8]:
records_pd = pd.DataFrame.from_dict(records).T

In [9]:
records_pd.to_excel('pubmed_ath_llm_qwen25.xlsx')